In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
import random
import json
import pickle
import numpy as np

import nltk
from nltk.stem import WordNetLemmatizer

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD

In [3]:
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [4]:
lemmatizer = WordNetLemmatizer()

In [5]:
intents = json.loads(open('/content/drive/MyDrive/intents.json').read())

In [6]:
words = []
classes = []
documents = []
ignore_letters = ['?', ',', '.', '!', "'"]

In [7]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [8]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [9]:
words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]
words = sorted(set(words))

In [10]:
classes = sorted(set(classes))

In [11]:
pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

In [12]:
training = []
output_empty = [0] * len(classes)

for document in documents:
    bag = []
    word_patterns = document[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)

        output_row = list(output_empty)
        output_row[classes.index(document[1])] = 1
        training.append([bag, output_row])

In [13]:
random.shuffle(training)
training = np.array(training)

train_x = list(training[:, 0])
train_y = list(training[:, 1])

<ipython-input-13-b5d0b17a3307>:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [14]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics='accuracy')

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [15]:
hist = model.fit(np.array(train_x), np.array(train_y), epochs=300, batch_size=8, verbose=1)

Epoch 1/300
2253/2253 [==============================] - 5s 2ms/step - loss: 0.9471 - accuracy: 0.7151
Epoch 2/300
2253/2253 [==============================] - 4s 2ms/step - loss: 0.3166 - accuracy: 0.8862
Epoch 3/300
2253/2253 [==============================] - 4s 2ms/step - loss: 0.2626 - accuracy: 0.8995
Epoch 4/300
2253/2253 [==============================] - 4s 2ms/step - loss: 0.2416 - accuracy: 0.9014
Epoch 5/300
2253/2253 [==============================] - 4s 2ms/step - loss: 0.2312 - accuracy: 0.9054
Epoch 6/300
2253/2253 [==============================] - 4s 2ms/step - loss: 0.2251 - accuracy: 0.9055
Epoch 7/300
2253/2253 [==============================] - 4s 2ms/step - loss: 0.2197 - accuracy: 0.9042
Epoch 8/300
2253/2253 [==============================] - 4s 2ms/step - loss: 0.2180 - accuracy: 0.9063
Epoch 9/300
2253/2253 [==============================] - 4s 2ms/step - loss: 0.2155 - accuracy: 0.9073
Epoch 10/300
2253/2253 [==============================] - 4s 2ms/step - l

In [16]:
model.save('chatbotmodel.h5', hist)
print("Done")

Done


In [17]:
from tensorflow.keras.models import load_model

In [18]:
words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl','rb'))
model = load_model('chatbotmodel.h5')

In [19]:
def clean_up_sentence(sentence):
  sentence_words = nltk.word_tokenize(sentence)
  sentence_words = [lemmatizer.lemmatize(word) for word in sentence_words]
  return sentence_words

In [20]:
def bag_of_words(sentence):
  sentence_words = clean_up_sentence(sentence)
  bag = [0] * len(words)
  for w in sentence_words:
    for i, word in enumerate(words):
      if word == w:
        bag[i] = 1
  return np.array(bag)

In [21]:
def predict_class(sentence):
  bow = bag_of_words(sentence)
  res = model.predict(np.array([bow]))[0]
  ERROR_THRESHOLD = 0.25
  results = [[i,r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]

  results.sort(key=lambda x: x[1], reverse=True)
  return_list = []
  for r in results:
    return_list.append({'intent': classes[r[0]], 'probability': str(r[1])})
  return return_list

In [22]:
def get_response(intents_list, intents_json):
  tag = intents_list[0]['intent']
  list_of_intents = intents_json['intents']
  for i in list_of_intents:
    if i['tag'] == tag:
      result = random.choice(i['responses'])
      break
  return result

In [ ]:
print("Go")

Go


In [27]:
while True:
  message = input("")
  ints = predict_class(message)
  res = get_response(ints, intents)
  print(res)

good morning
1/1 [==============================] - 0s 23ms/step
What's going on?
nothing much. eating food
1/1 [==============================] - 0s 18ms/step
I like to eat chips...
do you like other things?
1/1 [==============================] - 0s 22ms/step
Texas Chainsaw Massacre is my favorite horror movie of all time
why?
1/1 [==============================] - 0s 20ms/step
Goodbye
wait. do you watch anime?
1/1 [==============================] - 0s 18ms/step
Burt Kreischer
i am confused
1/1 [==============================] - 0s 23ms/step
Peace
tell me about crypto or data science
1/1 [==============================] - 0s 25ms/step
Data science is an interdisciplinary field that uses scientific methods, processes, algorithms and systems to extract or extrapolate knowledge and insights from noisy, structured and unstructured data
where did you study?
1/1 [==============================] - 0s 19ms/step
Acting
where?
1/1 [==============================] - 0s 17ms/step
Preserving oxyge

KeyboardInterrupt: ignored